In [ ]:
import csv
from scraper import WebsiteScraper

def debug_response(scraper:WebsiteScraper, debug=False):
  summary: str = ''
  if not scraper.connectivity_report:
    raise('You must first call test_website_connectivity()')
  
  try:
    if not scraper.connectivity_report.is_domain_active:
      summary = "[Domain inactive]"
    elif not scraper.connectivity_report.is_server_reachable:
      summary = "[Website server unreachable]"
    else: 
      
      redirect = ''
      if debug and scraper.connectivity_report.is_redirecting:
        redirect = scraper.connectivity_report.redirect_report.redirected_domain
        print(f"Redirect: {scraper._domain} -> {redirect}")
      
      if not scraper.connectivity_report.is_website_active:
        error = scraper.connectivity_report.redirect_report.error
        print(f"[Site inactive: {error}]")
        summary = ''
      elif scraper._text:
        if redirect: summary = f"[{redirect}] "
        summary += "OK"
      else:
        print(f"No text fetched for {scraper._domain}")
      
  except Exception as e:
    print(e)
    summary = ''
  finally:
    scraper._summary = summary
    return scraper._summary

def load_urls_from_csv(file_path, column_name):
  urls = []
  with open(file_path, mode='r') as file:
      csv_reader = csv.DictReader(file)
      for row in csv_reader:
          if column_name in row:
              urls.append(row[column_name])
  return urls

def get_summaries_from_websites(urls, debug=False):
  summaries = []
  
  for url in urls:
    summary = ''
    if url: 
      if debug: print (f"{url}")
      scraper = WebsiteScraper(url)
      scraper.scrape_text(debug)
      summary = debug_response(scraper, debug)
      if debug: print(f"{summary}\n")
        
    summaries.append(summary)
    
  return summaries

def save_results_to_csv(file_path, new_column_name, summaries):
    # Read existing data
    data = []
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        fieldnames = csv_reader.fieldnames + [new_column_name]  # add new column to fieldnames
        for row in csv_reader:
            data.append(row)
    
    # Append new column data
    for row, summary in zip(data, summaries):
        row[new_column_name] = summary
    
    # Write data back to CSV
    with open(file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

csv_path = 'data/organizations.csv'
urls = [
  'http://theblurapp.com/', #404
  'https://fullyfunctional.com.au/', #405
  'http://www.tri-implants.com', #timout OK
]
urls = load_urls_from_csv(csv_path, 'homepage_url')
results = get_summaries_from_websites(urls, True)
#save_results_to_csv(csv_path, 'result', results)